# SCIENCE

Of course, all is well and fun, and we can visualize some cool quantum operations on an 'inmage state', but obviously the main reason for these embeddings is not to make nice pictures, but to use them to encode data for tasks like classification for QML, or we can use image embedding to encode different types of data - linear depth is a pretty nice embedding ratio after-all! 

# QML with QPIXL embedding with classical autoencoder for image compression

So instead of using just a directly image loaded QML, we first pretrain a classical autoencoder and do a transfer learning approach. FOr this toy example we use a pretrained resnet18, but one could imagine refining model parameters to a dataset at the same time as the quantum weights! 

In [1]:
import time
import os
import copy

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms
# Import the relevant packages

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import cudaq
from cudaq import spin
# Pennylane
import pennylane as qml
from pennylane import numpy as np

torch.manual_seed(42)
np.random.seed(42)
import param_qpixl_qml as pq
from qiskit.circuit import ParameterVector
# Plotting
import matplotlib.pyplot as plt
# SETUP
# os.environ["OMP_NUM_THREADS"] = "8"
n_qubits = 11                # Number of qubits
step = 0.0004               # Learning rate
batch_size = 2            # Number of samples for each training step
num_epochs = 30              # Number of training epochs
q_depth = 6                 # Depth of the quantum circuit (number of variational layers)
gamma_lr_scheduler = 0.1    # Learning rate reduction applied every 10 epochs.
q_delta = 0.01              # Initial spread of random quantum weights
start_time = time.time()    # Start of the computation timer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
sample_count = 140

X_train = datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(),transforms.Lambda(lambda x: torch.flatten(x))]),
)

# Leaving only labels 0 and 1
idx = np.append(
    np.where(X_train.targets == 0)[0][:sample_count],
    np.where(X_train.targets == 1)[0][:sample_count],
)

X_train.data = X_train.data[idx]
X_train.targets = X_train.targets[idx]

train_loader = torch.utils.data.DataLoader(X_train, batch_size=1, shuffle=True)

# Test set
sample_count = 70

X_test = datasets.MNIST(
    root="./data",
    train=False,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(),transforms.Lambda(lambda x: torch.flatten(x))]),
)
idx = np.append(
    np.where(X_test.targets == 0)[0][:sample_count],
    np.where(X_test.targets == 1)[0][:sample_count],
)

X_test.data = X_test.data[idx]
X_test.targets = X_test.targets[idx]
X_test.data.reshape((X_test.data.shape[0],X_test.data.shape[1]**2))
test_loader = torch.utils.data.DataLoader(X_test, batch_size=1, shuffle=True)


    
dataloaders = {
    'train':train_loader,'test':test_loader
}
dataset_sizes = {x: len(dataloaders[x]) for x in ["train", "test"]}


## QNN 
We use a tree tensor network ansatz to try and do the classification

In [3]:
## QUANTUM NN
def block(weights, wires):
    qml.RX(weights[0], wires=wires[0])
    qml.RZ(weights[1], wires=wires[0])
    qml.RX(weights[2], wires=wires[1])
    qml.RZ(weights[3], wires=wires[1])
    qml.CNOT(wires=wires)
    
dev = qml.device("lightning.gpu", wires=n_qubits+5)
@qml.qnode(dev, interface="torch")
def quantum_net(input_feat,weights):
    quantum_input = pq.encode_image(input_feat)
    pq.param_qpixl(quantum_input) ## PENNYL:ANE QPIXL
    qml.TTN(wires=range(n_qubits+5),
            n_block_wires=2,
            block=block,
            n_params_block=4,
            template_weights=weights)
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in [14,15]]
    return exp_vals
def costfunc(params):
    cost = 0
    for i in range(len(BAS)):
        if i < len(BAS) / 2:
            cost += circuit(BAS[i], params)
        else:
            cost -= circuit(BAS[i], params)
    return cost


## Hybrid QNN with classical autoencoder
We use a classical autoencoder to reduce the dimensions in such a way that the image highlights features we want to classify with the QNN, where the embedding is done with QPIXL and the trained weights affect the tensor network tree. Our loss is cross entropy, the preffered method for classification tasks

In [4]:
class DressedQuantumNet(nn.Module):
    """
    Torch module implementing the *dressed* quantum net.
    """

    def __init__(self):
        """
        Definition of the *dressed* layout.
        """

        super().__init__()
        self.pre_net = nn.Linear(28*28,1)
        self.q_params = nn.Parameter(q_delta * torch.randn((15,4)))
        self.post_net = nn.Linear(2, 2)
        self.float()

    def forward(self, input_features):
        """
        Defining how tensors are supposed to move through the *dressed* quantum
        net.
        """

        # obtain the input features for the quantum circuit
        # by reducing the feature dimension from 512 to 4
        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(input_features) * np.pi / 2.0

        # Apply the quantum circuit to each element of the batch and append to q_out
        q_out = torch.Tensor(0, 2)
        q_out = q_out.to(device)
        q_in = q_in.to(torch.float64)
        for elem in q_in:
            q_out_elem = [quantum_net(elem, self.q_params)]
            q_out_elem = torch.tensor(q_out_elem,requires_grad=True).to(device)
            q_out = torch.cat((q_out,q_out_elem))
            q_out = q_out.to(torch.float)
            # q_out[0] = q_out_elem[0]
            # q_out[1] = q_out_elem[1]

            
        # return the two-dimensional prediction from the postprocessing layer
        return self.post_net(q_out)


In [5]:
# for param in model_hybrid.parameters():
#     param.requires_grad = False
model_hybrid = DressedQuantumNet()
model_hybrid = model_hybrid.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_hybrid = optim.Adam(model_hybrid.parameters(), lr=step)
exp_lr_scheduler = lr_scheduler.StepLR(
    optimizer_hybrid, step_size=10, gamma=gamma_lr_scheduler
)

def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 10000.0  # Large arbitrary number
    best_acc_train = 0.0
    best_loss_train = 10000.0  # Large arbitrary number
    print("Training started:")

    for epoch in range(num_epochs):

        # Each epoch has a training and validation phase
        for phase in ["train", "test"]:
            if phase == "train":
                # Set model to training mode
                model.train()
            else:
                # Set model to evaluate mode
                model.eval()
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            n_batches = dataset_sizes[phase] // batch_size
            it = 0
            for inputs, labels in dataloaders[phase]:
                since_batch = time.time()
                batch_size_ = len(inputs)
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                # Track/compute gradient and make an optimization step only when training
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # Print iteration results
                running_loss += loss.item() * batch_size_
                batch_corrects = torch.sum(preds == labels.data).item()
                running_corrects += batch_corrects
                print(
                    "Phase: {} Epoch: {}/{} Iter: {}/{} Batch time: {:.4f}".format(
                        phase,
                        epoch + 1,
                        num_epochs,
                        it + 1,
                        n_batches + 1,
                        time.time() - since_batch,
                    ),
                    end="\r",
                    flush=True,
                )
                it += 1

            # Print epoch results
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            print(
                "Phase: {} Epoch: {}/{} Loss: {:.4f} Acc: {:.4f}        ".format(
                    "train" if phase == "train" else "test  ",
                    epoch + 1,
                    num_epochs,
                    epoch_loss,
                    epoch_acc,
                )
            )

            # Check if this is the best model wrt previous epochs
            if phase == "test" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == "test" and epoch_loss < best_loss:
                best_loss = epoch_loss
            if phase == "train" and epoch_acc > best_acc_train:
                best_acc_train = epoch_acc
            if phase == "train" and epoch_loss < best_loss_train:
                best_loss_train = epoch_loss

            # Update learning rate
            if phase == "train":
                scheduler.step()

    # Print final results
    model.load_state_dict(best_model_wts)
    time_elapsed = time.time() - since
    print(
        "Training completed in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60)
    )
    print("Best test loss: {:.4f} | Best test accuracy: {:.4f}".format(best_loss, best_acc))
    return model


In [ ]:
model_hybrid = train_model(
    model_hybrid, criterion, optimizer_hybrid, exp_lr_scheduler, num_epochs=num_epochs
)


Training started:


/usr/local/lib/python3.10/dist-packages/pennylane/math/utils.py:227: UserWarning: Contains tensors of types {'autograd', 'torch'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


Phase: train Epoch: 1/30 Loss: 0.6953 Acc: 0.5000        
Phase: test   Epoch: 1/30 Loss: 0.6947 Acc: 0.5000        
Phase: train Epoch: 2/30 Loss: 0.6947 Acc: 0.5000        
Phase: test   Epoch: 2/30 Loss: 0.6942 Acc: 0.5000        
